In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-147526
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-147526


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster-demo"

if cpu_cluster_name in ws.compute_targets:
    cpu_cluster = ws.compute_targets[cpu_cluster_name]
    print("Using existing cluster: {}".format(cpu_cluster_name))
else:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("Creating new compute cluster")

cpu_cluster.wait_for_completion(show_output=True)


Using existing cluster: cpu-cluster-demo
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        "--C": uniform(0,1),
        "max_iter": choice(50, 100)
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
#hyperdrive_run.wait_for_completion(show_output=True)
#assert(hyperdrive_run.get_status() == 'Completed')
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_model = hyperdrive_run.get_best_run_by_primary_metric()
best_model_metrics = best_model.get_metrics()
best_model_parameter_values = best_model.get_details()['runDefinition']['arguments']
print(best_model_metrics, best_model_parameter_values)

#model = best_model.register_model(model_name='project1')

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [8]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_ouput=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_30a5997f-c56a-4b54-8187-ab61a2e4b982,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_model, fitted_model = automl_run.get_output()

In [10]:
best_automl_model.get_metrics()

{'f1_score_macro': 0.774234920077976,
 'f1_score_micro': 0.917754119417864,
 'recall_score_macro': 0.7515439266025542,
 'average_precision_score_weighted': 0.9565385856164573,
 'precision_score_weighted': 0.9120109770497948,
 'precision_score_micro': 0.917754119417864,
 'average_precision_score_macro': 0.8289163988969411,
 'f1_score_weighted': 0.9138993104238192,
 'norm_macro_recall': 0.5030878532051084,
 'precision_score_macro': 0.8046209093113554,
 'balanced_accuracy': 0.7515439266025542,
 'recall_score_micro': 0.917754119417864,
 'accuracy': 0.917754119417864,
 'log_loss': 0.17336204140646058,
 'weighted_accuracy': 0.9590669300580058,
 'AUC_micro': 0.9811361037545203,
 'recall_score_weighted': 0.917754119417864,
 'AUC_weighted': 0.948834812758932,
 'matthews_correlation': 0.5533839931353315,
 'AUC_macro': 0.948834812758932,
 'average_precision_score_micro': 0.9819343077141064,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_30a5997f-c56a-4b54-8187-ab61a2e4b982_44/co